<center> 
<h2><b> Adnan Rashid </b></h2>
    <nav>
        <h3>
    | <a href="00-Contents-Setup.ipynb"> Home Page </a> | 
        <a href="http://bit.ly/cka_notes_original"> Kubernetes Notes </a> |
        <a href="https://adnan.study"> Website </a> |
        <a href="https://www.instagram.com/adnans_techie_studies/"> Instagram </a> |
        <a href="https://www.linkedin.com/in/adnanrashid1/"> LinkedIn </a> |
        </h3>
    </nav>
</center>

<h1 style="background:black"><code style="background:black;color:white"><center>Advanced VPC Networking with EKS</center></code></h1>

<div class="alert alert-block alert-warning">
<font style="color:black">
<center>We will review some of the advanced VPC networking features with EKS </center>
</font>
</div>

<center><img src="https://adnanstudyimages.s3-eu-west-1.amazonaws.com/24.jpg" width="50%" height="50%" border=5/></center>

## <br><br> Using Secondary CIDRS with EKS 

* You can expand your VPC network by adding additonal CIDR ranges. 
* This capability can be used if you are running out of IP ranges within your existing VPC or if you have confusmed all available RFC 1918 CIDR ranges within your coporate network. 
* EKS supports additional IPv4 CIDR blocks in the 100.64.0.0/10 and 198.19.0.0/16 ranges

Below we go through the configuration that is needed so that you can launch your Pod networking on top of secondary CIDRs

## Pre-requisite

* Check your variables are loaded

In [ ]:
%store

* Load in variable store

In [ ]:
%store -r 

* Load variables into notebook

In [ ]:
%env AWS_PROFILE=$AWS_PROFILE
%env AWS_DEFAULT_REGION=$AWS_DEFAULT_REGION
%env ACCOUNT_ID=$ACCOUNT_ID

* Check identity 

In [ ]:
!aws sts get-caller-identity

<br>

---
---

<br>

Before configuring EKS, we need to enable secondary CIDR blocks in the VPC and make sure they have proper tags and route table configurations

#### Add secondary CIDRs to your VPC 

In [ ]:
!aws ec2 describe-vpcs --filters Name=tag:Name,Values=eksctl-eksworkshop-eksctl-cluster/VPC | jq -r '.Vpcs[].VpcId'

In [ ]:
%env VPC_ID=vpc-0a56b9642c5e016e0

In [ ]:
!aws ec2 associate-vpc-cidr-block --vpc-id $VPC_ID --cidr-block 100.64.0.0/16 

* Next step is to create subnets. Before we do this step, lets check how many subnets we are consuming. 
* We can run this command to see EC2 instnace and AZ details

In [ ]:
!aws ec2 describe-instances --filters "Name=tag-value,Values=eksworkshop*"  --query 'Reservations[*].Instances[*].[PrivateDnsName,Tags[?Key==`eks:nodegroup-name`].Value|[0],Placement.AvailabilityZone,PrivateIpAddress,PublicIpAddress]' --output table   

* Currently using 3 instances in 3 subnets 
* We will use the same AZ's and create 3 secondary CIDR subnets 

* It is easier to do as a single block so will comment in the code block

In [ ]:
%%bash 

# Set the AZ variables 
export AZ1=eu-west-1a
export AZ2=eu-west-1b
export AZ3=eu-west-1c

# Get the Subnet ID for each for the subnets 
export CGNAT_SNET1=$(aws ec2 create-subnet --cidr-block 100.64.0.0/19 --vpc-id $VPC_ID --availability-zone $AZ1 | jq -r .Subnet.SubnetId)
export CGNAT_SNET2=$(aws ec2 create-subnet --cidr-block 100.64.32.0/19 --vpc-id $VPC_ID --availability-zone $AZ2 | jq -r .Subnet.SubnetId)
export CGNAT_SNET3=$(aws ec2 create-subnet --cidr-block 100.64.64.0/19 --vpc-id $VPC_ID --availability-zone $AZ3 | jq -r .Subnet.SubnetId)

# We will need this later for cleanup 
echo "%env CGNAT_SNET1=$CGNAT_SNET1"
echo "%env CGNAT_SNET2=$CGNAT_SNET2"
echo "%env CGNAT_SNET3=$CGNAT_SNET3"

# Create the tags on each of the subnets 

aws ec2 create-tags --resources $CGNAT_SNET1 --tags Key=eksctl.cluster.k8s.io/v1alpha1/cluster-name,Value=eksworkshop-eksctl 
aws ec2 create-tags --resources $CGNAT_SNET1 --tags Key=kubernetes.io/cluster/eksworkshop-eksctl,Value=shared 
aws ec2 create-tags --resources $CGNAT_SNET1 --tags Key=kubernetes.io/role/elb,Value=1 
aws ec2 create-tags --resources $CGNAT_SNET2 --tags Key=eksctl.cluster.k8s.io/v1alpha1/cluster-name,Value=eksworkshop-eksctl 
aws ec2 create-tags --resources $CGNAT_SNET2 --tags Key=kubernetes.io/cluster/eksworkshop-eksctl,Value=shared 
aws ec2 create-tags --resources $CGNAT_SNET2 --tags Key=kubernetes.io/role/elb,Value=1
aws ec2 create-tags --resources $CGNAT_SNET3 --tags Key=eksctl.cluster.k8s.io/v1alpha1/cluster-name,Value=eksworkshop-eksctl 
aws ec2 create-tags --resources $CGNAT_SNET3 --tags Key=kubernetes.io/cluster/eksworkshop-eksctl,Value=shared 
aws ec2 create-tags --resources $CGNAT_SNET3 --tags Key=kubernetes.io/role/elb,Value=1 

# Now we need to associate the three new subnets into a route table. 

export SNET1=$(aws ec2 describe-subnets --filters Name=cidr-block,Values=192.168.0.0/19 | jq -r '.Subnets[].SubnetId')
export RTASSOC_ID=$(aws ec2 describe-route-tables --filters Name=association.subnet-id,Values=$SNET1 | jq -r '.RouteTables[].RouteTableId')

aws ec2 associate-route-table --route-table-id $RTASSOC_ID --subnet-id $CGNAT_SNET1 
aws ec2 associate-route-table --route-table-id $RTASSOC_ID --subnet-id $CGNAT_SNET2 
aws ec2 associate-route-table --route-table-id $RTASSOC_ID --subnet-id $CGNAT_SNET3

* Saving the variables for later

In [ ]:
%env CGNAT_SNET1=subnet-0e1f237091c482a0e
%env CGNAT_SNET2=subnet-01cbfaea3a30f84ca
%env CGNAT_SNET3=subnet-0f44727dd6e52f401

## <br><br> Configure CNI

* Before we start making changes to VPC CNI, let's make sure we are using latest CNI version

In [ ]:
!kubectl describe daemonset aws-node --namespace kube-system | grep Image | cut -d "/" -f 2

* upgrade to latest version if necesssary

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/aws/amazon-vpc-cni-k8s/master/config/v1.6/aws-k8s-cni.yaml

* Check status of pods

In [ ]:
!kubectl get pods -n kube-system 

#### <br><br>Configure Custom Networking

* Edit aws-node configmap and add <b>'AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG'</b> environment variable to the node container spec and set it to true
* You only need to set one environment variable in the CNI daemonset configuration 

In [ ]:
!kubectl set env ds aws-node -n kube-system AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG=true

In [ ]:
!kubectl describe daemonset aws-node -n kube-system | grep -A5 Environment 

* Terminate worker nodes so that Autoscaling launches newer nodes that come bootstrapped with custom network config

In [ ]:
%%bash 

INSTANCE_IDS=(`aws ec2 describe-instances --query 'Reservations[*].Instances[*].InstanceId' --filters "Name=tag-value,Values=eksworkshop-eksctl" --output text` )
for i in "${INSTANCE_IDS[@]}"
do
	echo "Terminating EC2 instance $i ..."
	aws ec2 terminate-instances --instance-ids $i
done

* Check nodes if they are back, you may need to wait 5-10 minutes 

In [ ]:
!kubectl get nodes

## <br><br>Create CRDS

#### Create custom resources for ENIConfig CRD 

* We will add custom resources to ENIConfig custom resource definition (CRD)
* CRDs are extensions of Kubernetes API that stores collection of API objects of a certain kind
* In this case, we will store VPC Subnet and SecurityGroup configuration information in these CRDs so that Worker nodes can access them to configure VPC CNI plugin

* Check ENIConfig CRD

In [ ]:
!kubectl get crd

* If the ENIConfig is not installed, can install with this command

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/aws/amazon-vpc-cni-k8s/master/config/v1.3/aws-k8s-cni.yaml

* Create custom resources for each subnet by replacing Subnet and SecurityGroup IDs
* Since we created three secondary subnets, we need three custom resources 

Here is the template for custom resource.

* Subnet ID and SecurityGroup ID need to be replaced with appropriate values in the CRD folder shortly 

* Check the AZs and Subnet IDs for these subnets

In [ ]:
!aws ec2 describe-subnets  --filters "Name=cidr-block,Values=100.64.*" --query 'Subnets[*].[CidrBlock,SubnetId,AvailabilityZone]' --output table

* Check Worker Node SecurityGroup

In [ ]:
%%bash 

INSTANCE_IDS=(`aws ec2 describe-instances --query 'Reservations[*].Instances[*].InstanceId' --filters "Name=tag-value,Values=eksworkshop*" --output text`)
for i in "${INSTANCE_IDS[@]}"
do
  echo "SecurityGroup for EC2 instance $i ..."
  aws ec2 describe-instances --instance-ids $i  --profile gr-sandbox-admin --region eu-west-1 | jq -r '.Reservations[].Instances[].SecurityGroups[].GroupId'
done  

* In crd/group1-pod-netconfig.yaml for the first subnet <b>(100.64.0.0/19)</b> replace the SubnetID and SecurityGroupIds with the values from above 

In [ ]:
!cat crd/group1-pod-netconfig.yaml

* We need to do similar for the second and third subnet, you can use the same security groups
    * CRD/group2-pod-netconfig.yaml for the second subnet <b>(100.64.32.0/19)</b>
    * CRD/group3-pod-netconfig.yaml for the third subnet <b> (100.64.64.0/19)</b> 

In [ ]:
!cat crd/group2-pod-netconfig.yaml

In [ ]:
!cat crd/group3-pod-netconfig.yaml

* Check the instance details using the below as <b>you will need AZ info when you apply annotation to Worker nodes</b> using custom network config

In [ ]:
!aws ec2 describe-instances --filters "Name=tag-value,Values=eksworkshop*" --query 'Reservations[*].Instances[*].[PrivateDnsName,Tags[?Key==`eks:nodegroup-name`].Value|[0],Placement.AvailabilityZone,PrivateIpAddress,PublicIpAddress]' --output table

* Apply the CRDs

In [ ]:
!kubectl apply -f CRD/group1-pod-netconfig.yaml 
!kubectl apply -f CRD/group2-pod-netconfig.yaml 
!kubectl apply -f CRD/group3-pod-netconfig.yaml

* Annotate nodes with custom network configs 
* Make sure the subnets align, you will need to check above the subnet the instance and the config 

In [ ]:
!kubectl annotate node ip-192-168-62-249.eu-west-1.compute.internal k8s.amazonaws.com/eniConfig=group1-pod-netconfig
!kubectl annotate node ip-192-168-83-85.eu-west-1.compute.internal k8s.amazonaws.com/eniConfig=group2-pod-netconfig
!kubectl annotate node ip-192-168-9-192.eu-west-1.compute.internal k8s.amazonaws.com/eniConfig=group3-pod-netconfig

* If the coredns pods are not in a ready state, then someone has gone wrong above, check the security groups and the subnets 

In [ ]:
!kubectl get pods -n kube-system 

## <br><br>Test Networking 

#### Launch pods into Secondary CIDR network 

* Let's launch few pods and test networking

In [ ]:
!kubectl apply -f https://k8s.io/examples/controllers/nginx-deployment.yaml

In [ ]:
!kubectl scale deployment nginx-deployment --replicas=3 

In [ ]:
!kubectl expose deployment nginx-deployment --type=NodePort --port 80

In [ ]:
!kubectl get pods -o wide

* Test access to the internet and to nginx service (Will need to run in console as interactive)

<p style="background:black"><code style="background:black;color:white">> kubectl run -i --rm --tty debug --image=busybox -- sh</code>
</p>

* Test access to internet and to nginx service

<p style="background:black"><br>
<code style="background:black;color:white">> wget google.com -O - <br><br></code>
<code style="background:black;color:white">> wget nginx-deployment -O - <br><br></code>
</p>

## <br><br> Clean Up

In [ ]:
!kubectl delete deployments --all 

* Comment out 'AWS_VPC_K8S_CNI_CUSTOM_NETWORK_CFG' and its value

<p style="background:black">
<code style="background:black;color:white">> kubectl edit daemonset -n kube-system aws-node
</code>
</p> 

* Delete custom resource objects from ENIConfig CRD

In [ ]:
!kubectl delete eniconfig/group1-pod-netconfig
!kubectl delete eniconfig/group2-pod-netconfig
!kubectl delete eniconfig/group3-pod-netconfig

* Terminate EC2 instances so fresh instances are launched with default CNI configuration

In [ ]:
%%bash

INSTANCE_IDS=(`aws ec2 describe-instances --query 'Reservations[*].Instances[*].InstanceId' --filters "Name=tag-value,Values=eksworkshop*" --output text`)
for i in "${INSTANCE_IDS[@]}"
do
	echo "Terminating EC2 instance $i ..."
	aws ec2 terminate-instances --instance-ids $i 
done

* Delete secondary CIDR from your VPC, you may need to get the values from above

In [ ]:
%%bash 

VPC_ID=$(aws ec2 describe-vpcs --filters Name=tag:Name,Values=eksctl-eksworkshop* | jq -r '.Vpcs[].VpcId')

aws ec2 delete-subnet --subnet-id $CGNAT_SNET1
aws ec2 delete-subnet --subnet-id $CGNAT_SNET2
aws ec2 delete-subnet --subnet-id $CGNAT_SNET3

* Get Association ID

In [ ]:
!aws ec2 describe-vpcs --vpc-id $VPC_ID | jq -r '.Vpcs[].CidrBlockAssociationSet[] | select(.CidrBlock == "100.64.0.0/16") | .AssociationId'

* Disassociate the secondary CIDR block

In [ ]:
!aws ec2 disassociate-vpc-cidr-block --association-id vpc-cidr-assoc-030d456d7bddbc0d0

* Configure CNI again

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/aws/amazon-vpc-cni-k8s/master/config/v1.6/aws-k8s-cni.yaml